<a href="https://colab.research.google.com/github/sgcortes/legion-del-mal/blob/master/VISIONT4_Practica2_CNN_CLASIFICACION_SENTINEL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
  <tr>
    <td><img src="https://unioviedo-my.sharepoint.com/:i:/g/personal/sgcortes_uniovi_es/Ebr0ybKDTBNKtoJyzmTBuRYBXY7nE-cCYaH-WDIVJxEVWA?&download=1" width="211" height="69" alt="Uniovi & EP Mieres logos" title="Uniovi & EP Mieres logos" /></td>
    <td><font color=brown>Visión artificial<br></font>
    <font color=green>Universidad de Oviedo. <br>Ingeniería en Geomática</font> <br><br>sgcortes@uniovi.es</td>
  </tr>
</table>

### PRACTICA 4.2: CLASIFICACIONES DE CUBIERTA MEDIANTE CNN EN IMÁGENES SENTINEL2 (CNN) 

In [26]:
#!pip install tensorflow==2.2.0

##!wget https://unioviedo-my.sharepoint.com/personal/sgcortes_uniovi_es/_layouts/15/download.aspx?UniqueId=98537e25%2D6b87%2D48f1%2Dbb65%2D959d4fd8b2ed
#https://unioviedo-my.sharepoint.com/personal/sgcortes_uniovi_es/_layouts/15/download.aspx?UniqueId=98537e25%2D6b87%2D48f1%2Dbb65%2D959d4fd8b2ed

!wget https://unioviedo-my.sharepoint.com/:u:/g/personal/sgcortes_uniovi_es/ESV-U5iHa_FIu2WVnU_Ysu0BLFb1BX1hzvzTFbivXLj1ZA?e=GOfJP9&download=1
!mv view?usp=sharing EuroSAT.zip
!unzip -qq EuroSAT.zip
%cd EuroSAT/2756/

--2020-12-14 16:57:48--  https://unioviedo-my.sharepoint.com/:u:/g/personal/sgcortes_uniovi_es/ESV-U5iHa_FIu2WVnU_Ysu0BLFb1BX1hzvzTFbivXLj1ZA?e=GOfJP9
Resolving unioviedo-my.sharepoint.com (unioviedo-my.sharepoint.com)... 13.107.136.9
Connecting to unioviedo-my.sharepoint.com (unioviedo-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unioviedo-my.sharepoint.com/personal/sgcortes_uniovi_es/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fsgcortes%5Funiovi%5Fes%2FDocuments%2F2020%5FVISION%2FEuroSAT%2Ezip&parent=%2Fpersonal%2Fsgcortes%5Funiovi%5Fes%2FDocuments%2F2020%5FVISION&originalPath=aHR0cHM6Ly91bmlvdmllZG8tbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvc2djb3J0ZXNfdW5pb3ZpX2VzL0VTVi1VNWlIYV9GSXUyV1ZuVV9Zc3UwQkxGYjFCWDFoenZ6VEZiaXZYTGoxWkE_cnRpbWU9aHVRZFkxR2cyRWc [following]
--2020-12-14 16:57:48--  https://unioviedo-my.sharepoint.com/personal/sgcortes_uniovi_es/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fsgcortes%5Funiovi%

In [ ]:
S%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import keras

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
dade4300


ValueError: ignored

#### Carga de datos 
Realizamos la carga de datos y recordemos las clases posibles que coicniden con los nombres de los subdirectorios.

In [ ]:
ruta_datos = '../EuroSAT/2750/'
clases = os.listdir(ruta_datos)
print(clases)

['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


Inicializamos un diccionario para asociar las clases a un número y unas listas que contengan las imágenes de entrada y las etiquetas de clase en el vector _datos de salida_.  

In [ ]:
dic_clases = {}
datos_entrada = []
datos_salida = []
i=0

for i, clase in enumerate(clases):
    if os.path.isdir(ruta_datos+clase):
        for file in os.listdir(ruta_datos+clase):
            im = Image.open(os.path.join(ruta_datos, clase, file))
            im_array = np.array(im)
            datos_entrada.append(im_array)
            datos_salida.append(i)
        dic_clases[i] = clase
        
datos_entrada = np.array(datos_entrada)
datos_salida = np.array(datos_salida)

print(datos_entrada.shape) # num imgs, anchura, altura, canaales
print(datos_salida.shape) # etiqueta de clase de cada imagen

(27000, 64, 64, 3)
(27000,)


In [ ]:
print(np.unique(datos_salida))

[0 1 2 3 4 5 6 7 8 9]


#### División de datos de entrenamiento y test

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, clases_train, clases_test = train_test_split(datos_entrada,
                                                           datos_salida,
                                                           train_size=0.8,
                                                           random_state=1,
                                                           shuffle=True)
print('Dimensión imágenes entrada x_train: ', x_train.shape)
print('Dimension clases train: ', clases_train.shape)
# convertir la salida a foramto "one-hot"
y_train = keras.utils.to_categorical(clases_train)
print('Dimension y train: ', y_train.shape)
y_test = keras.utils.to_categorical(clases_test)

Dimensión imágenes entrada x_train:  (21600, 64, 64, 3)
Dimension clases train:  (21600,)
Dimension y train:  (21600, 10)


In [ ]:
# Normalización de imágenes
x_train = x_train/255
x_test =x_test/255

In [ ]:
# Comprobamos el tamaño del conjnto de datos
print(x_train.shape)
print(x_test.shape)

(21600, 64, 64, 3)
(5400, 64, 64, 3)


In [ ]:
print(y_train.shape)
print(y_test.shape)

(21600, 10)
(5400, 10)


### Caso 1: CNN, Convolutional Neural Networks
A efectos de comparación de las métricas de desempeño podemos emplear como punto de comparación un modelo CNN como el que sigue, sin MaxPooling2D, ni dropout ni Batch Normalization.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D
from keras.layers import MaxPooling2D, Flatten

cnn = Sequential()
cnn.add(Conv2D(16, 5, strides = (2,2),
              activation='relu',
              padding = 'same',
              input_shape = (64,64,3)))

cnn.add(Conv2D(32, 3, strides = (2,2),
              activation='relu',
              padding = 'same'))

cnn.add(Conv2D(64, 3, strides = (2,2),
              activation='relu',
              padding = 'same'))

cnn.add(Flatten())       
cnn.add(Dense(50, activation = 'relu'))
cnn.add(Dense(10, activation = 'softmax'))
cnn.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 16)        1216      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                204850    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 229,712
Trainable params: 229,712
Non-trainable params: 0
_____________________________________________

Compilamos y entrenamos el modelo:

In [ ]:
cnn.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
hist = cnn.fit(x_train, y_train,
               epochs =25,
               batch_size =128,
               validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 21600 samples, validate on 5400 samples
Epoch 1/25
21600/21600 [==============================] - 7s 304us/step - loss: 1.6333 - acc: 0.3920 - val_loss: 1.3316 - val_acc: 0.5144
Epoch 2/25
21600/21600 [==============================] - 4s 193us/step - loss: 1.2135 - acc: 0.5691 - val_loss: 1.1767 - val_acc: 0.6063
Epoch 3/25
21600/21600 [==============================] - 4s 184us/step - loss: 1.0084 - acc: 0.6482 - val_loss: 0.9146 - val_acc: 0.6680
Epoch 4/25
21600/21600 [==============================] - 4s 184us/step - loss: 0.8564 - acc: 0.7021 - val_loss: 0.8143 - val_acc: 0.7106
Epoch 5/25
21600/21600 [==============================] - 4s 184us/step - loss: 0.7951 - acc: 0.7197 - val_loss: 0.7877 - val_acc: 0.7161
Epoch 6/25
21600/21600 [==============================] - 4s 189us/step - loss: 0.7323 - acc: 0.7414 - val_loss: 0.7628 - val_acc: 0.7335
Epoch 7/25
21600/21600 [====

## MaxPooling
Cambiemos el modelo añadiendo MaxPooling. Recordemos que de este modo disminuimos la dimensiones de los mapas (con lo que podemos aumentar el número de canales) y añadimos regularización. Además disminuye mucho el número de parámetros a entrenar.

In [ ]:
cnn2 = Sequential()
cnn2.add(Conv2D(16, 5, strides = (2,2),
              activation='relu',
              padding = 'same',
              input_shape = (64,64,3)))
cnn2.add(MaxPooling2D(pool_size=2))

cnn2.add(Conv2D(32, 3, strides = (2,2),
              activation='relu',
              padding = 'same'))
cnn2.add(MaxPooling2D(pool_size=2))

cnn2.add(Conv2D(64, 3, strides = (2,2),
              activation='relu',
              padding = 'same'))
cnn2.add(MaxPooling2D(pool_size=2))

cnn2.add(Flatten())       
cnn2.add(Dense(50, activation = 'relu'))
cnn2.add(Dense(10, activation = 'softmax'))
cnn2.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 16)        1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)              

In [ ]:
cnn2.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
hist = cnn2.fit(x_train, y_train,
               epochs =25,
               batch_size =128,
               validation_data=(x_test, y_test))

Train on 21600 samples, validate on 5400 samples
Epoch 1/25
21600/21600 [==============================] - 4s 202us/step - loss: 1.7527 - acc: 0.3445 - val_loss: 1.3813 - val_acc: 0.5131
Epoch 2/25
21600/21600 [==============================] - 4s 181us/step - loss: 1.2590 - acc: 0.5448 - val_loss: 1.2199 - val_acc: 0.5544
Epoch 3/25
21600/21600 [==============================] - 4s 167us/step - loss: 1.1141 - acc: 0.5965 - val_loss: 1.0746 - val_acc: 0.6057
Epoch 4/25
21600/21600 [==============================] - 4s 170us/step - loss: 1.0482 - acc: 0.6103 - val_loss: 0.9871 - val_acc: 0.6335
Epoch 5/25
21600/21600 [==============================] - 4s 171us/step - loss: 1.0009 - acc: 0.6267 - val_loss: 1.0270 - val_acc: 0.6143
Epoch 6/25
21600/21600 [==============================] - 4s 169us/step - loss: 0.9346 - acc: 0.6535 - val_loss: 0.9147 - val_acc: 0.6567
Epoch 7/25
21600/21600 [==============================] - 4s 178us/step - loss: 0.9144 - acc: 0.6603 - val_loss: 0.9924 - v

### Red mas profunda 
También es posible hacer la red mas profunda:

In [ ]:
cnn3 = Sequential()
cnn3.add(Conv2D(16, 5,activation='relu',padding = 'same',input_shape = (64,64,3)))
cnn3.add(Conv2D(16, 5, strides = (2,2),activation='relu',padding = 'same',input_shape = (64,64,3)))
cnn3.add(MaxPooling2D(pool_size=2))

cnn3.add(Conv2D(32, 3,activation='relu',padding = 'same'))
cnn3.add(Conv2D(32, 3, strides = (2,2),
              activation='relu',
              padding = 'same'))
cnn3.add(MaxPooling2D(pool_size=2))

cnn3.add(Conv2D(64, 3,activation='relu',padding = 'same'))
cnn3.add(Conv2D(64, 3,activation='relu',padding = 'same'))
cnn3.add(MaxPooling2D(pool_size=2))

cnn3.add(Conv2D(128, 3,activation='relu',padding = 'same'))
cnn3.add(Conv2D(128, 3,activation='relu',padding = 'same'))
cnn3.add(MaxPooling2D(pool_size=2))

cnn3.add(Flatten()) 
cnn3.add(Dropout(0.4))
cnn3.add(Dense(50, activation = 'relu'))
cnn3.add(Dense(10, activation = 'softmax'))
cnn3.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 16)        1216      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 16)        6416      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 32)          0         
_____

In [ ]:
cnn3.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
hist = cnn3.fit(x_train, y_train,
               epochs =25,
               batch_size =128,
               validation_data=(x_test, y_test))

Train on 21600 samples, validate on 5400 samples
Epoch 1/25
21600/21600 [==============================] - 7s 308us/step - loss: 1.8631 - acc: 0.2539 - val_loss: 1.6573 - val_acc: 0.3333
Epoch 2/25
21600/21600 [==============================] - 5s 251us/step - loss: 1.4360 - acc: 0.4424 - val_loss: 1.2089 - val_acc: 0.5346
Epoch 3/25
21600/21600 [==============================] - 5s 248us/step - loss: 1.1392 - acc: 0.5824 - val_loss: 1.1705 - val_acc: 0.5556
Epoch 4/25
21600/21600 [==============================] - 5s 241us/step - loss: 0.9850 - acc: 0.6442 - val_loss: 0.8784 - val_acc: 0.6861
Epoch 5/25
21600/21600 [==============================] - 5s 253us/step - loss: 0.8854 - acc: 0.6837 - val_loss: 0.8019 - val_acc: 0.7054
Epoch 6/25
21600/21600 [==============================] - 6s 271us/step - loss: 0.7765 - acc: 0.7204 - val_loss: 0.7595 - val_acc: 0.7359
Epoch 7/25
21600/21600 [==============================] - 6s 263us/step - loss: 0.7148 - acc: 0.7457 - val_loss: 0.6917 - v

## Batch Normalization
Es una técnica que normaliza las entradas de las funciones de activación. Mejora la estabilidad del proceso, un poco la regularización aunque aumenta ligeramente el número de parámetros a entrenar. 

In [ ]:
from keras.layers import BatchNormalization

cnn4 = Sequential()
cnn4.add(Conv2D(16, 5,activation='relu',padding = 'same',input_shape = (64,64,3)))
cnn4.add(BatchNormalization())
cnn4.add(Conv2D(16, 5,activation='relu',padding = 'same',input_shape = (64,64,3)))
cnn4.add(MaxPooling2D(pool_size=2))

cnn4.add(Conv2D(32, 3,activation='relu',padding = 'same'))
cnn4.add(BatchNormalization())
cnn4.add(Conv2D(32, 3,activation='relu',padding = 'same'))
cnn4.add(MaxPooling2D(pool_size=2))

cnn4.add(Conv2D(64, 3,activation='relu',padding = 'same'))
cnn4.add(BatchNormalization())
cnn4.add(Conv2D(64, 3,activation='relu',padding = 'same'))
cnn4.add(MaxPooling2D(pool_size=2))

cnn4.add(Conv2D(128, 3,activation='relu',padding = 'same'))
cnn4.add(BatchNormalization())
cnn4.add(Conv2D(128, 3,activation='relu',padding = 'same'))
cnn4.add(MaxPooling2D(pool_size=2))

cnn4.add(Flatten()) 
cnn4.add(Dropout(0.4))
cnn4.add(Dense(50, activation = 'relu'))
cnn4.add(Dense(10, activation = 'softmax'))
cnn4.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 64, 64, 16)        1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 16)        64        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 64, 16)        6416      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 32)      

In [ ]:
cnn4.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
hist4 = cnn4.fit(x_train, y_train,
               epochs =25,
               batch_size =256,
               validation_data=(x_test, y_test))

Train on 21600 samples, validate on 5400 samples
Epoch 1/25
21600/21600 [==============================] - 10s 462us/step - loss: 1.0452 - acc: 0.6376 - val_loss: 2.1827 - val_acc: 0.4833
Epoch 2/25
21600/21600 [==============================] - 8s 350us/step - loss: 0.6147 - acc: 0.7832 - val_loss: 0.8168 - val_acc: 0.7213
Epoch 3/25
21600/21600 [==============================] - 8s 371us/step - loss: 0.5215 - acc: 0.8153 - val_loss: 1.0537 - val_acc: 0.7007
Epoch 4/25
21600/21600 [==============================] - 8s 358us/step - loss: 0.4292 - acc: 0.8488 - val_loss: 3.8040 - val_acc: 0.3419
Epoch 5/25
21600/21600 [==============================] - 7s 338us/step - loss: 0.3755 - acc: 0.8713 - val_loss: 0.9155 - val_acc: 0.7463
Epoch 6/25
21600/21600 [==============================] - 8s 352us/step - loss: 0.3258 - acc: 0.8854 - val_loss: 1.1114 - val_acc: 0.6902
Epoch 7/25
21600/21600 [==============================] - 7s 347us/step - loss: 0.2797 - acc: 0.9044 - val_loss: 0.9167 - 